# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[0])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
['', 'Logged In', 'Kevin', 'M', '0', 'Arellano', '', 'free', 'Harrisburg-Carlisle, PA', 'GET', 'Home', '1.54001E+12', '514', '', '200', '1.54207E+12', '66']


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
df = pd.read_csv('event_datafile_new.csv')
df.columns

Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
       'level', 'location', 'sessionId', 'song', 'userId'],
      dtype='object')

In [9]:
df.dropna().head(1)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Fu,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",514,Ja I Ty,66


In [10]:
df.head(1)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Fu,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",514,Ja I Ty,66


#### CREATE table

In [11]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
drop_song_history = "DROP TABLE IF EXISTS song_history"
session.execute(drop_song_history)

create_song_history = '''
    CREATE TABLE iF NOT EXISTS song_history
    (session_id int, item_in_session int, artist text, song text, length float, PRIMARY KEY(session_id, item_in_session))
'''

session.execute(create_song_history)          

In [12]:
df.head(1)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Fu,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",514,Ja I Ty,66


#### Insert Data

In [13]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    # insert data into song_history
        query = "INSERT INTO song_history(session_id, item_in_session, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        # print(line[8], line[3], line[0], line[9], line[5])
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
select_from_song_history = '''
    SELECT * FROM song_history WHERE session_id=514 AND item_in_session=1
'''

rows = session.execute(select_from_song_history)
print(rows[0])

Row(session_id=514, item_in_session=1, artist='Fu', length=280.0583190917969, song='Ja I Ty')


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [15]:
df.head(1)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Fu,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",514,Ja I Ty,66


#### CREATE Table

In [17]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
drop_user_history = "DROP TABLE IF EXISTS user_history"

session.execute(drop_user_history)

create_user_history = '''
    CREATE TABLE iF NOT EXISTS user_history
    (user_id int, session_id int, item_in_session int,  
    artist text, first_name text, last_name text, song text, 
    PRIMARY KEY(user_id, session_id, item_in_session))
'''

session.execute(create_user_history)

In [18]:
df.head(1)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Fu,Kevin,M,1,Arellano,280.05832,free,"Harrisburg-Carlisle, PA",514,Ja I Ty,66


#### INSERT DATA

In [19]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    # insert data into song_history
        query = '''INSERT INTO user_history(user_id, 
        session_id, item_in_session, artist, 
        first_name, last_name, song)'''
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [20]:
#  userid = 10, sessionid = 182
select_from_song_history = '''
    SELECT * FROM user_history WHERE user_id=66 AND session_id=514
'''

rows = session.execute(select_from_song_history)
print(rows[0])

Row(user_id=66, session_id=514, item_in_session=1, artist='Fu', first_name='Kevin', last_name='Arellano', song='Ja I Ty')


#### CREATE Table

In [21]:
# TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
drop_user_songs = "DROP TABLE IF EXISTS user_songs"

session.execute(drop_user_songs)

create_song_history = '''
    CREATE TABLE iF NOT EXISTS user_songs
    (song text, user_id int, first_name text, last_name text, PRIMARY KEY(song, user_id))
'''

session.execute(create_song_history)   

#### INSERT DATA

In [22]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
    # insert data into song_history
        query = "INSERT INTO user_songs(song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [23]:
select_user_songs = '''
    SELECT * FROM user_songs WHERE song='All Hands Against His Own'
'''

rows = session.execute(select_user_songs)

for row in rows:
    print(row)

Row(song='All Hands Against His Own', user_id=29, first_name='Jacqueline', last_name='Lynch')
Row(song='All Hands Against His Own', user_id=80, first_name='Tegan', last_name='Levine')
Row(song='All Hands Against His Own', user_id=95, first_name='Sara', last_name='Johnson')


### Drop the tables before closing out the sessions

In [27]:
drop_song_history = "DROP TABLE IF EXISTS song_history"
user_history = "DROP TABLE IF EXISTS user_history"
user_songs = "DROP TABLE IF EXISTS user_songs"
session.execute(drop_song_history)
session.execute(user_history)
session.execute(user_songs)

### Close the session and cluster connection¶

In [29]:
session.shutdown()
cluster.shutdown()